In [1]:
import pickle
import numpy as np
from collections import Counter

In [2]:
number_of_games = 100000
filename = "/home/jaoi/master22/pet_for_sale/winning_games_db/3PPO_{}_games.pkl".format(number_of_games)
# with open("/home/jaoi/master22/pet_for_sale/winning_games_db/{}_games.pkl".format(number_of_games), "rb") as fp:
with open(filename, "rb") as fp:
        games = pickle.load(fp)


In [3]:
games[0].shape

(49,)

In [3]:
def convert_3_rounds_fs1_obs_to_bin_obs(obs):
    stack_len = 11
    rounds = 3
    
    coin_numbers = [0, 5, 6, 9, 14, 15, 18, 23, 24]
    board_card_numbers = [1,2,3,7,10,11,12,16,19,20,21,25,38,39,40]
    can_bet_numbers = [4,13,22]
    order_numbers = [8,17,26]
    std_number = 41
    last_stack_nr = 37
    bin_rep = []
    onehot_of_stack = np.zeros(stack_len, dtype="float32")
    for i, num in enumerate(obs):
        if i in coin_numbers:
            bin_rep.extend(list("{0:05b}".format(int(num)))) 
        elif i in board_card_numbers:
            bin_rep.extend(list("{0:04b}".format(int(num))))
        elif i in can_bet_numbers:
            bin_rep.append(int(num))
        elif i in order_numbers:
            bin_rep.extend(list("{0:02b}".format(int(num))))
        elif i == std_number:
            if num > (np.std([1,11]) - np.std([1])) / 2:
                bin_rep.append(1)
            else:
                bin_rep.append(0)
            
        else:
            if i == last_stack_nr:
                if int(num) != 0:
                    onehot_of_stack[-int(num)] = 1
                    bin_rep.extend(list(onehot_of_stack))
                else:
                    bin_rep.extend(list(onehot_of_stack))
                    
            else:
                if int(num) != 0:
                    onehot_of_stack[-int(num)] = 1
    bin_rep = np.hstack(bin_rep)
    return bin_rep.astype(np.float32)

In [5]:
def convert_4_rounds_fs1_obs_to_bin_obs(obs):
    # 4 rounds, so hands also need to have 4 cards ... 
    stack_len = 15
    rounds = 4
    # Should be 9
    coin_numbers = [0, 6, 7, 10, 16, 17, 20, 26, 27]
    
    board_card_numbers = [1,2,3,4,8,11,12,13,14,18,21,22,23,24,28,45,46,47]
    can_bet_numbers = [5,15,25]
    order_numbers = [9,19,29]
    std_number = 48
    last_stack_nr = order_numbers[-1] + stack_len
    bin_rep = []
    onehot_of_stack = np.zeros(stack_len, dtype="float32")
    for i, num in enumerate(obs):
        if i in coin_numbers:
            bin_rep.extend(list("{0:05b}".format(int(num))))
        elif i in board_card_numbers:
            bin_rep.extend(list("{0:04b}".format(int(num))))
        elif i in can_bet_numbers:
            bin_rep.append(int(num))
        elif i in order_numbers:
            bin_rep.extend(list("{0:02b}".format(int(num))))
        elif i == std_number:
            if num > (np.std([1,stack_len]) - np.std([1])) / 2:
                bin_rep.append(1)
            else:
                bin_rep.append(0)
            
        else:
            if i == last_stack_nr:
                if int(num) != 0:
                    onehot_of_stack[-int(num)] = 1
                    bin_rep.extend(list(onehot_of_stack))
                else:
                    # onehot_of_stack[-int(num)] = 1
                    bin_rep.extend(list(onehot_of_stack))
            else:
                if int(num) != 0:
                    onehot_of_stack[-int(num)] = 1
    # print(onehot_of_stack)
    # print(obs[30:45], len(obs[30:45]))
    bin_rep = np.hstack(bin_rep)
    return bin_rep.astype(np.float32)

In [6]:
coin_numbers = [0, 5, 6, 9, 14, 15, 18, 23, 24]
board_card_numbers = [1,2,3,7,10,11,12,16,19,20,21,25,38,39,40]
can_bet_numbers = [4,13,22]
order_numbers = [8,17,26]
std_number = 41
s = set()
s.update(coin_numbers)
s.update(board_card_numbers)
s.update(can_bet_numbers)
s.update(order_numbers)
s.add(std_number)
s

print(len(coin_numbers) * 5 + len(board_card_numbers) * 4 + len(can_bet_numbers) * 1 + len(order_numbers) * 2 + 1 + 11)
print(len(coin_numbers) * 5 + len(board_card_numbers) * 4 + len(can_bet_numbers) * 1 + len(order_numbers) * 2 + 1 + 11)
# for i in range(42):
#     if i not in s:
#         print(i)

onehot = np.zeros(11, dtype="float32")
for i in [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]:
    
    onehot[-int(i)] = 1
    print(i, onehot)


126
126
1 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
2 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1.]
3 [0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1.]
4 [0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1.]
5 [0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1.]
6 [0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1.]
7 [0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1.]
8 [0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1.]
9 [0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
10 [0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
11 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


In [7]:
print(convert_4_rounds_fs1_obs_to_bin_obs(games[0][0]).shape)
print(convert_4_rounds_fs1_obs_to_bin_obs(games[4][0]).shape)     

(142,)
(142,)


In [4]:
c = Counter()
tsetlinified_games = []
for i, game in enumerate(games):
    tset_game = convert_3_rounds_fs1_obs_to_bin_obs(game[0])
    c[tset_game.shape] += 1
    
    # print(tset_game.shape)
    # print(np.where(tset_game>1))
    # print(tset_game)
    
    
    tsetlinified_games.append((tset_game, game[1]))

# print(tsetlinified_games)
print(c)
tsetlinified_games = np.array(tsetlinified_games, dtype=object)
# tsetlinified_games = np.hstack(tsetlinified_games, dtype=object)
# print(tsetlinified_games[0][0])
# print(type(tsetlinified_games))



Counter({(126,): 552271})


In [6]:
tsetlin_filename = "/home/jaoi/master22/pet_for_sale/winning_games_db/3PPO_{}_tsetlined_games.pkl".format(number_of_games)
with open(tsetlin_filename, "wb") as fp:
        pickle.dump(tsetlinified_games, fp)

In [10]:
tsetlinified_games[0][0].shape

(142,)